# Topic Modeling - Actividad

Topic modeling es una técnica de aprendizaje automático no supervisado donde intentados descubrir tópicos que son abstractos al texto pero que pueden describir una colección de documentos. Es importante marcar que estos "tópicos" no son necesariamente equivalentes a la interpretación coloquial de tópicos, sino que responden a un patrón que emerge de las palabras que están en los documentos.

La suposición básica para Topic Modeling es que cada documento está representado por una mescla de tópicos, y cada tópico consite en una conlección de palabras.

## Direcciones
Intentaremos construir un pipeline de machine learning donde como entrada recibamos texto, ejecutemos todos los pasos que vimos en este notebook incluyendo:
 - Eliminación de stopwords
 - Tokenización
 - Stemming y Lemmatization
 - Procesamiento especico del tema
 - Creación de features utilizando algun metodo de reducción de dimensionalidad, SVD, LSI, LDA

, para luego utilizar estas features para entrenar un modelo que nos permita predecir alguna propiedad interesante del set de datos. En este caso en particular, donde estamos viendo tweets, algunos casos interesantes podrían ser:
 - Predecir el sector al que pertenece el tweet: Alimentación, Bebidas, etc.
 - Predecir el paso en el Marketing Funel al que pertece
 
En esta actividad les propongo realizar cambios en alguna de las etapas del procesamiento para modificar la performance del modelo resultante

<img src='https://github.com/santiagxf/M72109/blob/master/NLP/Docs/atap_0406.png?raw=1' />

## Modelado

Descargamos el set de datos y lo cargmamos en un DataFrame

In [ ]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv --directory-prefix ./Datasets/mascorpus/

In [7]:
import pandas as pd

tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

### Instalamos las librerias necesarias

In [ ]:
!pip install unidecode
!python -m spacy download es_core_news_sm

### Construccion de los pasos

**Paso 1:** Instanciamos nuestro preprocesamiento de texto

In [1]:
import unidecode
import spacy
import es_core_news_sm as spa
import re
import sklearn
from nltk import stem
from nltk.corpus import stopwords
from nltk.tokenize.casual import TweetTokenizer

class TextNormalizer(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin):
    def __init__(self, language='spanish'):
        self.parser = spa.load() # Cargamos el parser en español
        self.tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True) # Creamos un tokenizer
        self.stemmer = stem.SnowballStemmer(language=language) # Creamos un steammer
        self.lemmatizer = lambda word : " ".join([token.lemma_ for token in self.parser(word)]) # Creamos un lemmatizer
        self.stopwords = set(stopwords.words(language)) # Instanciamos las stopwords en español
        self.urls_regex = re.compile('http\S+') # Usamos una expresion regular para encontrar las URLs
    
    def process_text(self, text):
        tokens = self.tokenizer.tokenize(text)
        tokens = [token for token in tokens if not re.match(urls_regex, token)]
        tokens = [token for token in tokens if len(token) > 4]
        tokens = [token for token in tokens if token not in self.stopwords]
        tokens = [unidecode.unidecode(token) for token in tokens] # Quitamos acentos
        tokens = [self.lemmatizer(token) for token in tokens]
        return tokens
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        for doc in X:
          yield ' '.join(self.process_text(text=doc))

In [2]:
normalizer = TweetTextNormalizer()

**Paso 2:** Instanciamos nuestro vectorizador, en este caso usando el método TF-IDF

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=True, sublinear_tf=True, norm='l2')

**Paso 3:** Instanciamos nuestro generador de features

In [4]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF

featurizer = LatentDirichletAllocation(n_components=7)

**Paso 4:** Instanciamos nuestro clasificador que utilizará las features generadas hasta este momento

In [5]:
from sklearn.linear_model import LogisticRegression

estimator = LogisticRegression(max_iter=10000, multi_class='multinomial')

Ensamblamos el pipeline

In [6]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(steps=[('normalizer', normalizer), 
                           ('vectorizer', vectorizer),
                           ('featurizer', featurizer),
                           ('estimator', estimator)])

### Evaluación

**Evaluación:** Entrenamos el modelo y testeamos su performance

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets['TEXTO'], tweets['SECTOR'], test_size=0.33, stratify=tweets['SECTOR'])

In [9]:
model = pipeline.fit(X=X_train, y=y_train)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions))